# Entry & Exit Signal

- Create separate notebook as 'poc.ipynb' notebook is too bloated.
- Create abstract class


In [12]:
import pandas as pd
from abc import ABC, abstractmethod

%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.utils import utils
from config.variables import EntryType, COINT_CORR_FN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
date = "2025-04-01"
model = "ziweichen_coint_5"
ticker_pair = "NFLX_EA"

results_dir = f"./data/results/{date}"
model_dir = f"{results_dir}/{model}"
pa_dir = f"{model_dir}/price_actions"

In [6]:
df = utils.load_csv(f"{pa_dir}/{ticker_pair}.csv")
df

,date,ticker,day_name,is_holiday,av_rating,median_rating_excl,NFLX_close,EA_close,action
0,2025-03-01,NFLX,Saturday,False,3.622950819672131,5.0,NaN,NaN,buy
1,2025-03-02,NFLX,Sunday,False,5.0,5.0,NaN,NaN,buy
2,2025-03-03,NFLX,Monday,False,3.0,3.0,973.7,129.23,wait
3,2025-03-04,NFLX,Tuesday,False,3.0,3.0,972.58,131.82,wait
4,2025-03-05,NFLX,Wednesday,False,3.25,4.0,990.92,134.05,buy
5,2025-03-06,NFLX,Thursday,False,4.5,5.0,906.36,136.79,buy
6,2025-03-07,NFLX,Friday,False,3.0,3.0,891.11,140.04,wait
7,2025-03-08,NFLX,Saturday,False,2.75,2.5,NaN,NaN,wait
8,2025-03-09,NFLX,Sunday,False,3.0,3.0,NaN,NaN,wait
9,2025-03-10,NFLX,Monday,False,5.0,5.0,866.68,140.43,buy


In [19]:
a = df.iloc[:, :-1].copy()

a["day_name"] == "Monday"

0     False
1     False
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16     True
17    False
18    False
19    False
20    False
21    False
22    False
23     True
24    False
25    False
26    False
27    False
28    False
29    False
30     True
31    False
Name: day_name, dtype: bool